In [11]:
import sys
sys.path.append("../src/")
import gc

from util import *
from FE import add_features_choice
from dataset import read_data, prepare_train_valid
from infer_helper import get_test_avg, get_cv_score
from config import read_config, update_config, prepare_args
from models import get_model

In [3]:
config = read_config("base_fc128")
config.gpu = [0]
config.train_folds = [0]
config = update_config(config)

Using device: cuda
Number of device: 1
Model Output Folder: /home/vincent/Kaggle/GBVPP/output/base_fc128/


In [4]:
train, test = read_data(config)
del test
train = add_features_choice(train.copy(), config)
seed_torch(seed=config.seed)

--- Generate u_out features ---
--- Generate time features ---
--- Generate u_in features ---
--- Generate cross sectional features ---
--- Generate R C features ---


In [9]:
_,_,_, X_valid, y_valid, w_valid = prepare_train_valid(train, config, 0)

Prepare train valid
(6036000, 63)
Use scale to fit train and scale valid


In [13]:
model = get_model(X_valid.shape[-1], config)
print("Model Size: {}".format(get_n_params(model)))
model.to(config.device)

Model Size: 5811457


Model(
  (lstms): ModuleList(
    (0): LSTM(59, 512, batch_first=True, bidirectional=True)
    (1): LSTM(1024, 256, batch_first=True, bidirectional=True)
    (2): LSTM(512, 128, batch_first=True, bidirectional=True)
    (3): LSTM(256, 64, batch_first=True, bidirectional=True)
  )
  (fc1): Linear(in_features=128, out_features=128, bias=True)
  (act): SELU()
  (fc2): Linear(in_features=128, out_features=1, bias=True)
  (scaler): ScaleLayer()
)

In [15]:
COLS = [col for col in train.columns if col not in ["id", "breath_id", "fold", "pressure"]]

In [16]:
results = []
print(' Computing LSTM feature importance...')
# COMPUTE BASELINE (NO SHUFFLE)
oof_preds = model(X_valid, verbose=0).squeeze() 
baseline_mae = (np.abs( oof_preds-y_valid ) * w_valid).sum()/w_valid.sum()
results.append({'feature':'BASELINE','mae':baseline_mae})           
for k in tqdm(range(len(COLS))):
    # SHUFFLE FEATURE K
    save_col = X_valid[:,:,k].copy()
    np.random.shuffle(X_valid[:,:,k])

    # COMPUTE OOF MAE WITH FEATURE K SHUFFLED
    oof_preds = model.predict(X_valid, verbose=0).squeeze() 
    mae = np.mean(np.abs( oof_preds-y_valid ))
    results.append({'feature':COLS[k],'mae':mae})
    X_valid[:,:,k] = save_col

# DISPLAY LSTM FEATURE IMPORTANCE
print()
df = pd.DataFrame(results)
df = df.sort_values('mae')
plt.figure(figsize=(10,20))
plt.barh(np.arange(len(COLS)+1),df.mae)
plt.yticks(np.arange(len(COLS)+1),df.feature.values)
plt.title('LSTM Feature Importance',size=16)
plt.ylim((-1,len(COLS)+1))
plt.plot([baseline_mae,baseline_mae],[-1,len(COLS)+1], '--', color='orange',
         label=f'Baseline OOF\nMAE={baseline_mae:.3f}')
plt.xlabel(f'Fold 0 OOF MAE with feature permuted',size=14)
plt.ylabel('Feature',size=14)
plt.legend()
plt.show()

# SAVE LSTM FEATURE IMPORTANCE
df = df.sort_values('mae',ascending=False)
df.to_csv(f'../output/lstm_feature_importance_fold_0.csv',index=False)

 Computing LSTM feature importance...


AttributeError: 'Model' object has no attribute 'predict'